# Preprocesamiento del set de datos

Se preparará el set de datos en 2 etapas:

1) Analisis
2) Procesamiento

En la etapa de analisis se pretende determinar que datos son relevantes y cuales no, y determinar que datos textuales pueden ser transformados a datos numericos y como se realizara esta "traduccion". 

En la etapa de procesamiento se realizaran los cambios necesarios al set de datos.





In [1]:
import pandas as pd

dataset = pd.read_csv("Accidente.csv", parse_dates=[5,6,23], true_values = ['true','True'], false_values = ['false','False'])


In [2]:
dataset.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
dataset.dtypes

ID                               object
Source                           object
TMC                             float64
Severity                          int64
Start_Time                       object
End_Time                 datetime64[ns]
Start_Lat                        object
Start_Lng                       float64
End_Lat                         float64
End_Lng                         float64
Distance(mi)                    float64
Description                      object
Number                          float64
Street                           object
Side                             object
City                             object
County                           object
State                            object
Zipcode                          object
Country                          object
Timezone                         object
Airport_Code                     object
Weather_Timestamp                object
Temperature(F)                   object
Wind_Chill(F)                   float64


In [4]:
#dataset['Severity'] = dataset['Severity'].astype('category')
dataset['Side'] = dataset['Side'].astype('category')
dataset['Wind_Direction'] = dataset['Wind_Direction'].astype('category')
dataset['Weather_Condition'] = dataset['Weather_Condition'].astype('category')
dataset['Sunrise_Sunset'] = dataset['Sunrise_Sunset'].astype('category')
dataset['Civil_Twilight'] = dataset['Civil_Twilight'].astype('category')
dataset['Nautical_Twilight'] = dataset['Nautical_Twilight'].astype('category')
dataset['Astronomical_Twilight'] = dataset['Astronomical_Twilight'].astype('category')
dataset['Temperature(F)'] = dataset['Temperature(F)'].astype('float64')
dataset['Start_Time'] = dataset['Start_Time'].astype('datetime64')

#for i in range(32,49):
#    dataset[dataset.columns[i]] = dataset[dataset.columns[i]].astype('category')

In [5]:
time_elapsed = dataset["End_Time"] - dataset["Start_Time"] #Nos sirve mas el tiempo total que el momento de inicio y el de fin

In [6]:
dataset["Time_Elapsed"] = time_elapsed #Entonces creamos una nueva columna 

In [7]:
dataset = dataset.drop("ID",axis="columns") #
dataset = dataset.drop("Source",axis="columns") #No aporta info sobre la severidad
dataset = dataset.drop("TMC",axis="columns") #No aporta mucha informacion sobre la severidad
dataset = dataset.drop("Country",axis="columns") #Hay un solo valor en todas las columnas
dataset = dataset.drop("Description",axis="columns") #Partes de la descripcion se encuentran en otras columnas
dataset = dataset.drop("End_Lat",axis="columns") #No aportan mucha info
dataset = dataset.drop("End_Lng",axis="columns") #No aportan mucha info
dataset = dataset.drop("Number",axis="columns") #Tiene mas valores nulos que medidos
dataset = dataset.drop("Start_Time",axis="columns") #Ya guardamos el la duracion del accidente
dataset = dataset.drop("End_Time",axis="columns") #Ya guardamos la duracion del accidente
dataset = dataset.drop("Weather_Timestamp",axis="columns") #No coincide con el horario del accidente, por lo que no es relevante
dataset = dataset.drop("Airport_Code",axis="columns") #No aportan mucha info
dataset = dataset.drop("Timezone",axis="columns") #No aportan mucha info
dataset = dataset.drop("Zipcode",axis="columns") #No aportan mucha info
dataset = dataset.drop("State",axis="columns") #No aportan mucha info
dataset = dataset.drop("County",axis="columns") #No aportan mucha info


dataset.shape

(2974335, 34)

### Explicacion de la eliminacion de ciertas columnas

La logica por detras de la eleccion es la siguiente: los accidentes de transito son un fenomeno local. Los factores que suelen influenciar mas la severidad de un accidente suelen ser el estado del clima, si la via donde ocurren esta bien señalizada, etc.. 

Como fenomeno local queremos implicar que los factores que influyen sobre el accidente deben estar relativamente cercanos al
mismo, como por ejemplo un cruce sin semaforo, un cartel publicitario que distrae, asfalto mojado/congelado, etc.. Estos son fenomenos que se pueden observar a ojo desde el epicentro del accidente por lo que informacion sobre el codigo de aeropuerto mas cercano o el departamento del estado donde ocurrio el accidente no es relevante a la hora de determinar la severidad.

Un factor que si podemos correlacionar seria la calle, la altura e inclusive la ciudad. Una metropolis suele tener mas accidentes debido a la cantidad de autos que circulan o una calle a determinada altura tiene un pozo que hace que ocurran mas accidentes de lo normal. 

In [8]:
dataset.dtypes

Severity                           int64
Start_Lat                         object
Start_Lng                        float64
Distance(mi)                     float64
Street                            object
Side                            category
City                              object
Temperature(F)                   float64
Wind_Chill(F)                    float64
Humidity(%)                      float64
Pressure(in)                     float64
Visibility(mi)                   float64
Wind_Direction                  category
Wind_Speed(mph)                  float64
Precipitation(in)                float64
Weather_Condition               category
Amenity                             bool
Bump                                bool
Crossing                            bool
Give_Way                            bool
Junction                            bool
No_Exit                             bool
Railway                             bool
Roundabout                          bool
Station         

In [9]:
obj_cols = dataset.select_dtypes(['object']).columns
obj_cols.drop("Start_Lat")


Index(['Street', 'City'], dtype='object')

In [10]:
from sklearn.preprocessing import LabelEncoder

# define ordinal encoding
street_encoder = LabelEncoder()
# transform data
enc_street = street_encoder.fit_transform(dataset["Street"])
dataset["Street"] = enc_street

In [11]:
# define ordinal encoding
city_encoder = LabelEncoder()
# transform data
enc_city = city_encoder.fit_transform(dataset["City"])
dataset["City"] = enc_city

In [12]:
dataset.dtypes

Severity                           int64
Start_Lat                         object
Start_Lng                        float64
Distance(mi)                     float64
Street                             int64
Side                            category
City                               int64
Temperature(F)                   float64
Wind_Chill(F)                    float64
Humidity(%)                      float64
Pressure(in)                     float64
Visibility(mi)                   float64
Wind_Direction                  category
Wind_Speed(mph)                  float64
Precipitation(in)                float64
Weather_Condition               category
Amenity                             bool
Bump                                bool
Crossing                            bool
Give_Way                            bool
Junction                            bool
No_Exit                             bool
Railway                             bool
Roundabout                          bool
Station         

In [13]:
cat_columns = dataset.select_dtypes(['category']).columns
cat_columns

Index(['Side', 'Wind_Direction', 'Weather_Condition', 'Sunrise_Sunset',
       'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')

In [14]:
#Aca transformamos los valores de las columnas de datos categoricos a su representacion numerica

dataset = pd.get_dummies(dataset, columns=cat_columns) #get_dummies utiliza one hot encoding en las columnas categoricas

In [15]:
dataset.shape

(2974335, 182)

In [16]:
#%store dataset

In [18]:
dataset.to_feather("dataframe")